In [ ]:
%load_ext autoreload
%autoreload 2
import glob
import matplotlib.pyplot as plt
import cv2
import numpy as np
import pandas as pd

from ultralytics import YOLO

# Video to test trackreid on

### Let's try to detect jungling balls

In [ ]:
VIDEO_PATH = "../data/jungling_guy.mp4"

In [ ]:

from IPython.display import Video; Video(VIDEO_PATH, width=1024, height=600)


# Video to frames

In [ ]:
# Transform this VIDEO_PATH into a list of frames in this folder under frames/
!mkdir -p frames
!ffmpeg -i $VIDEO_PATH -vf fps=12 frames/%d.png -hide_banner -loglevel panic

In [ ]:
available_frames = glob.glob("frames/*.png")
available_frames = sorted(available_frames, key=lambda x: int(x.split("/")[-1].split(".")[0]))

# Detection

In [ ]:
%matplotlib inline
model = YOLO("yolov8n.pt")
results = model(available_frames[0])[0]

plt.imshow(cv2.cvtColor(results.plot(), cv2.COLOR_BGR2RGB))
plt.show()

### We will track only sports ball so we need to filter them out

In [ ]:
SPORTS_BALL_CLASS_ID = 32

## Tracking without trackreid

In [ ]:
from bytetracker import BYTETracker
from bytetracker.basetrack import BaseTrack
from utils import draw_all_bbox_on_image, yolo_results_to_bytetrack_format, scale_bbox_as_xyxy

In [ ]:
tracker = BYTETracker(track_thresh= 0.15, track_buffer = 3, match_thresh = 0.85, frame_rate= 12)
BaseTrack._count = 0
model = YOLO("yolov8n.pt", task="detect")

In [ ]:
all_tracked_objects  = []
for frame_id, image_filename in enumerate(available_frames):
    img = cv2.imread(image_filename)
    detections = model.predict(img, classes=[SPORTS_BALL_CLASS_ID], conf=0.15)[0]
    detections_bytetrack_format = yolo_results_to_bytetrack_format(detections)
    tracked_objects = tracker.update(detections_bytetrack_format, frame_id)
    if len(tracked_objects) > 0:
        tracked_objects = np.insert(tracked_objects, 0, frame_id, axis=1)
        all_tracked_objects.append(tracked_objects)

#### • Create a dataframe with the tracking informations

#### • Rescale the bbox to the original image size

In [ ]:
df_tracked = pd.DataFrame(np.concatenate(all_tracked_objects), columns=["frame_id", "x1", "y1", "x2", "y2", "track_id", "class", "confidence"])
df_tracked[["x1", "y1", "x2", "y2"]] = df_tracked[["x1", "y1", "x2", "y2"]].apply(
    lambda x: scale_bbox_as_xyxy(x[0:4], detections.orig_shape), axis=1, result_type="expand"
    )


### Show tracking video

In [ ]:
fourcc = cv2.VideoWriter_fourcc(*'mp4v') 
OUTPUT_WITH_BBOX = "videos/video_bbox_no_reid.mp4"
out = cv2.VideoWriter(OUTPUT_WITH_BBOX, fourcc, 12, (1280, 720))
for frame_id, image_filename in enumerate(available_frames):
    image = cv2.imread(image_filename)
    if frame_id in df_tracked.frame_id.astype('int').values:
        df_current_frame = df_tracked[df_tracked.frame_id == frame_id][["x1", "y1", "x2", "y2", "track_id", "class", "confidence"]].to_numpy()
        image = draw_all_bbox_on_image(image, df_current_frame)
    out.write(image)
out.release()


## Tracking with trackreid

In [ ]:
from trackreid.reid_processor import ReidProcessor


tracker = BYTETracker(track_thresh= 0.15, track_buffer = 3, match_thresh = 0.85, frame_rate= 12)
BaseTrack._count = 0
model = YOLO("yolov8n.pt", task="detect")

reid_processor = ReidProcessor(filter_confidence_threshold=0.1,
                                    filter_time_threshold=5,
                                    cost_function_threshold=5000, # max cost to rematch 2 objects
                                    max_attempt_to_match=5,
                                    max_frames_to_rematch=20,
                                    )

In [ ]:
all_tracked_objects  = []
for frame_id, image_filename in enumerate(available_frames):
    detections = model(image_filename, classes=[SPORTS_BALL_CLASS_ID])[0]
    detections_bytetrack_format = yolo_results_to_bytetrack_format(detections)
    tracked_objects = tracker.update(detections_bytetrack_format, frame_id)
    corrected_tracked_objects = reid_processor.update(tracked_objects, frame_id)
    if len(corrected_tracked_objects) > 0:
        all_tracked_objects.append(corrected_tracked_objects)
    

In [ ]:
df_tracked = pd.DataFrame(np.concatenate(all_tracked_objects), columns=["frame_id", "track_id", "class", "x1", "y1", "x2", "y2", "confidence", "mean_confidence", "orginal_track_id"])
df_tracked[["x1", "y1", "x2", "y2"]] = df_tracked[["x1", "y1", "x2", "y2"]].apply(
    lambda x: scale_bbox_as_xyxy(x[0:4], detections.orig_shape), axis=1, result_type="expand"
    )

In [ ]:
fourcc = cv2.VideoWriter_fourcc(*'mp4v') 
OUTPUT_WITH_BBOX = "videos/video_bbox_with_reid.mp4"
out = cv2.VideoWriter(OUTPUT_WITH_BBOX, fourcc, 12, (1280, 720))
for frame_id, image_filename in enumerate(available_frames):
    image = cv2.imread(image_filename)
    if frame_id in df_tracked.frame_id.astype('int').values:
        df_current_frame = df_tracked[df_tracked.frame_id == frame_id][["x1", "y1", "x2", "y2", "track_id", "class", "confidence"]].to_numpy()
        image = draw_all_bbox_on_image(image, df_current_frame)
    out.write(image)
out.release()
